In [2]:
import numpy as np
import os
import io
import matplotlib.pyplot as plt

## loading and saving data

In [18]:
images = []
labels = []

In [9]:
def load_images(directory, label):
    for filename in os.listdir(directory):
        file_path = directory + "/" + filename
        image = plt.imread(file_path)
        images.append(image)
        labels.append(label)

In [ ]:
load_images("./data/trash", 0)
load_images("./data/paper-recycling", 1)
load_images("./data/other-recycling", 2)

In [20]:
np.save("images.npy", np.array(images))
np.save("labels.npy", np.array(labels))

## Preprocessing and Memmap

In [3]:
images = np.load("./images.npy", mmap_mode="r")
labels = np.load("./labels.npy", mmap_mode="r")

In [8]:
images.shape

(2527, 384, 512, 3)

In [5]:
# compute mean
batch_size = int(len(labels) / 4)
sum_ = 0
for cnt in range(4):
    total = images[cnt * batch_size:(cnt + 1) * batch_size].sum(axis=0)
    sum_ += total
mean_images = sum_ / len(labels)

In [6]:
# compute standard deviation
total = 0
for cnt in range(4):
    total += np.sum((images[cnt * batch_size:(cnt + 1) * batch_size] - mean_images) ** 2, axis=0)
std_images = np.sqrt(total / len(labels))    

In [7]:
# save mean and std
np.save("./image_stats.npy", (np.array(mean_images), np.array(std_images)))

In [4]:
mean_images, std_images = np.load("./image_stats.npy")

In [6]:
# normalize all images
batch_size = int(len(labels) / 4)
normed_images = np.empty((2527, 3, 384, 512))
for cnt in range(4):
    normed_images[cnt * batch_size:(cnt + 1) * batch_size] = ((images[cnt * batch_size:(cnt + 1) * batch_size] - mean_images) / std_images).transpose(0,3,1,2)

In [13]:
# transpose to fix dimensions
normed_images = normed_images.transpose(0,3,1,2).copy()

memmap([[[[226, 198, 186],
          [226, 198, 186],
          [226, 198, 186],
          ...,
          [192, 172, 161],
          [192, 172, 161],
          [192, 172, 161]],

         [[226, 198, 186],
          [226, 198, 186],
          [226, 198, 186],
          ...,
          [193, 173, 162],
          [192, 172, 161],
          [192, 172, 161]],

         [[226, 198, 186],
          [226, 198, 186],
          [226, 198, 186],
          ...,
          [193, 173, 162],
          [193, 173, 162],
          [192, 172, 161]],

         ...,

         [[218, 191, 180],
          [218, 191, 180],
          [218, 191, 180],
          ...,
          [158, 139, 132],
          [157, 138, 131],
          [157, 138, 131]],

         [[218, 192, 179],
          [218, 192, 179],
          [218, 192, 179],
          ...,
          [158, 139, 132],
          [157, 138, 131],
          [157, 138, 131]],

         [[218, 192, 179],
          [218, 192, 179],
          [218, 192, 179],
         

In [7]:
# save normed images
np.save("./normed_images.npy", normed_images)

In [10]:
normed_images.shape

(2527, 384, 512, 3)

In [3]:
transposed_images = np.load("./transposed.npy", mmap_mode = "r")

In [4]:
transposed_images[0]

memmap([[[ 0.36677116,  0.3661065 ,  0.36512186, ...,  0.27039764,
           0.27238152,  0.27890733],
         [ 0.36729593,  0.36576731,  0.36500429, ...,  0.29203913,
           0.27121179,  0.27730878],
         [ 0.36736351,  0.3676344 ,  0.36444181, ...,  0.29260927,
           0.29622656,  0.27614072],
         ...,
         [ 0.33646927,  0.33946624,  0.33848228, ..., -0.0710815 ,
          -0.09063637, -0.0878033 ],
         [ 0.34042793,  0.33944773,  0.33982424, ..., -0.07376808,
          -0.0922981 , -0.08694377],
         [ 0.34239324,  0.34288857,  0.33954156, ..., -0.07556355,
          -0.09475904, -0.08929157]],

        [[-0.09278396, -0.09834652, -0.0998483 , ...,  0.01056183,
           0.01329295,  0.01935507],
         [-0.09282432, -0.09480796, -0.09548613, ...,  0.0314546 ,
           0.01129014,  0.01698485],
         [-0.09284611, -0.09244363, -0.09712148, ...,  0.03214142,
           0.03553001,  0.01598746],
         ...,
         [-0.10115274, -0.09909566

In [5]:
labels = np.load("./labels.npy")

## Train and test data split (not finished)

In [6]:
idxs = np.arange(len(labels))
np.random.shuffle(idxs)
idxs = idxs
train_indices = idxs[0: int(0.75 * len(labels))]
test_indices = idxs[int(0.75 * len(labels)):]

## Model and training

In [13]:
import torch
from torch import tensor
import torch.nn as nn
relu = nn.functional.relu
cross_entropy = nn.CrossEntropyLoss()

In [10]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        # MyNN's 'dense' layers are PyTorch's 'linear' layers.
        # Note that these layers do not accept weight-initialization functions.
        self.dense1 = nn.Linear(x_train.shape[1], 100)
        #need kernel size and stride
        self.CNN = nn.Conv2d(100, 50, 8, 8)
        self.dense3 = nn.Linear(50, 3)
        
        # We have to separately overwrite the weights in the layers with
        # the initialization scheme that we desire. Here, we use the
        # "xavier-normal" initialization scheme for the weights, and 
        # initialize the biases to 0
        # Note that these functions update the weights in-place; this 
        # is denoted, in pytorch, by the trailing underscore in the
        # function name: `xavier_normal_`
        # This is merely a convention adopted by PyTorch
        for layer in (self.dense1, self.CNN, self.dense3):
            nn.init.xavier_normal_(layer.weight, np.sqrt(2))
            nn.init.constant_(layer.bias, 0)    
        
    def forward(self, x):
        ''' Forward data through the network.
        
        This allows us to conveniently initialize a model `m` and then send data through it
        to be classified by calling `m(x)`.
        
        Parameters
        ----------
        x : Union[numpy.ndarray, mygrad.Tensor], shape=(N, D)
            The data to forward through the network.
            
        Returns
        -------
        mygrad.Tensor, shape=(N, 1)
            The model outputs.
        '''
        return self.dense3(relu(self.CNN(relu(self.dense1(x)))))
    
    
    
def accuracy(predictions, truth):
    """
    Returns the mean classification accuracy for a batch of predictions.
    
    Parameters
    ----------
    predictions : Union[numpy.ndarray, mg.Tensor], shape=(M, D)
        The scores for D classes, for a batch of M data points
    truth : numpy.ndarray, shape=(M,)
        The true labels for each datum in the batch: each label is an
        integer in [0, D)
    
    Returns
    -------
    float
    """
    if isinstance(predictions, mg.Tensor):
        predictions = predictions.data
    return np.mean((torch.argmax(prediction, dim=1) == truth).cpu().numpy())

In [9]:
# We will place these tensors in gpu memory, if a gpu is available, otherwise
# we will simply use the CPU. It is very unlikely that your laptop has a
# GPU for us to leverage.
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
device

device(type='cuda')

In [ ]:
from noggin import create_plot
plotter, fig, ax = create_plot(metrics=["loss", "accuracy"])

In [ ]:
# initialize our model and place its parameters on the GPU if it is available
model =  Model().to(device)

# initialize the optimizer
optim = torch.optim.Adam(model.parameters(), weight_decay=5E-3)

In [ ]:
num_epoch = 
batch_size = 

for epoch_cnt in range(num_epoch):
    for batch_cnt in range(len(train_indices) // batch_size):
        
        batch_x = transposed_images[train_indices[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]]
        batch_y = labels[train_indices[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]]
        
        x = torch.tensor(batch_x).to(device)
        y_true = torch.tensor(batch_y).to(device)
        y_pred = model(x)
        
        loss = cross_entropy(y_pred, y_true)
        loss.backward()
        optim.step()
        acc = accuracy(y_pred, y_true)
        
        optim.zero_grad()  # this is comparable to `null_gradients` in MyGrad
        plotter.set_train_batch({"loss" : loss.item(),
                                 "accuracy" : acc},
                                 batch_size=batch_size, plot=False)

        
        # test_data = torch.tensor(y_train, dtype=torch.int64).to(device)

        # X = torch.tensor(x_test).to(device)
        # Y = torch.tensor(y_test, dtype=torch.int64).to(device)